In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
import logging
import os
from ticker_data import get_df_for_period

#tickers = ['JNJ', '^SPX','KO','HD', 'TMO']
optimize_for_ticker = 'KO'
tickers = ['^SPX','KO']

# NDX is an index: volume=0, and High/Low/Open == Close (no intraday range)
#tickers_with_vol  = ['AAPL', '^SPX', '^NDX', 'MSFT']
#tickers_with_ohlc = ['AAPL', '^SPX', '^NDX', 'MSFT']

# NDX is an index: volume=0, and High/Low/Open == Close (no intraday range)
#tickers_with_vol  = ['AAPL', '^SPX', '^NDX', 'MSFT']
#tickers_with_ohlc = ['AAPL', '^SPX', '^NDX', 'MSFT']

period_train = {'start': '2020-06-01', 'end': '2025-01-01'}
period_test = {'start': '2025-01-01', 'end': '2025-09-01'}

df_train = get_df_for_period(tickers, period_train)
df_test = get_df_for_period(tickers, period_test)

print(f"Training endet am: {df_train['ds'].max()}")
print(f"Blind-Test startet am: {df_test['ds'].min()}")

#tickers = ['AAPL', 'SPY', 'MSFT', 'NDX']
# NDX is an index: volume=0, and High/Low/Open == Close (no intraday range)
#tickers_with_vol  = ['AAPL', 'SPY', 'MSFT']
#tickers_with_ohlc = ['AAPL', 'SPY', 'MSFT']


print(df_train['unique_id'].value_counts())
print(df_train.head(8))

[*********************100%***********************]  2 of 2 completed
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pandas/core/internals/blocks.py:347: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
[                       0%                       ]

unique_id
SPX_vol    2
unique_id
SPX_price    1154
SPX_open     1154
SPX_high     1154
SPX_low      1154
KO_price     1154
KO_open      1154
KO_high      1154
KO_low       1154
KO_vol       1154
SPX_vol      1152
Name: count, dtype: int64
Total days: 1154  |  Total rows: 11538  |  Series: 10
y range: [-1.489380, 1.625940]  |  mean: 0.000290  |  std: 0.119259


[*********************100%***********************]  2 of 2 completed

unique_id
SPX_price    164
SPX_open     164
SPX_high     164
SPX_low      164
SPX_vol      164
KO_price     164
KO_open      164
KO_high      164
KO_low       164
KO_vol       164
Name: count, dtype: int64
Total days: 164  |  Total rows: 1640  |  Series: 10
y range: [-1.328556, 1.069421]  |  mean: 0.000683  |  std: 0.117377
Training endet am: 2024-12-31 00:00:00
Blind-Test startet am: 2025-01-03 00:00:00
unique_id
SPX_price    1154
SPX_open     1154
SPX_high     1154
SPX_low      1154
KO_price     1154
KO_open      1154
KO_high      1154
KO_low       1154
KO_vol       1154
SPX_vol      1152
Name: count, dtype: int64
          ds  unique_id         y
0 2020-06-02  SPX_price  0.008177
1 2020-06-03  SPX_price  0.013557
2 2020-06-04  SPX_price -0.003374
3 2020-06-05  SPX_price  0.025874
4 2020-06-08  SPX_price  0.011970
5 2020-06-09  SPX_price -0.007830
6 2020-06-10  SPX_price -0.005327
7 2020-06-11  SPX_price -0.060753


In [ ]:
%%capture
import optuna
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.losses.pytorch import MAE
from pathlib import Path
from neuralforecast.losses.pytorch import MQLoss

N_FOLDS = 4
FOLD_STEP = 20  # trading days between fold cutoffs

study_id = "_".join(tickers) + '.' + period_train['start'] + '.' + period_train['end']
report_dir = "optuna_report"

rdir = Path(report_dir, study_id)
rdir.mkdir(parents=True, exist_ok=True)

df_train.to_csv(rdir / "traindata.csv", index=False, sep=",", header=True)


def objective(trial):
    model = PatchTST(
        h=7,
        input_size=trial.suggest_int('input_size', 20, 140, step=10),
        patch_len=trial.suggest_categorical('patch_len', [4, 8, 16]),
        stride=trial.suggest_categorical('stride', [2, 4, 8]),
        encoder_layers=trial.suggest_int('encoder_layers', 1, 5),
        n_heads=trial.suggest_categorical('n_heads', [2, 4, 8]),
        hidden_size=trial.suggest_categorical('hidden_size', [32, 64, 128]),
        linear_hidden_size=trial.suggest_categorical('linear_hidden_size', [32, 64, 128, 256, 512]),
        dropout=trial.suggest_float('dropout', 0.0, 0.3),
        learning_rate=trial.suggest_float('learning_rate', 1e-4, 1e-3, log=True),
        max_steps=500,
        loss=MQLoss(level=[80, 95]),
        val_check_steps=500,
        early_stop_patience_steps=-1,
        accelerator='gpu',
        devices=1,
        enable_progress_bar=False,
    )

    nf = NeuralForecast(models=[model], freq='D')

    all_dates = np.sort(df_train['ds'].unique())
    n_total = len(all_dates)

    fold_hit_rates = []
      
    for fold in range(N_FOLDS):
          cutoff = all_dates[n_total - FOLD_STEP * (N_FOLDS - fold)]
          df_fold = df_train[df_train['ds'] <= cutoff]

          cv = nf.cross_validation(df=df_fold, n_windows=3, step_size=1, refit=False)

          cv_target = cv[cv['unique_id'] == f'{optimize_for_ticker}_price']
          actual_dir = np.sign(cv_target['y'])

          # With MQLoss(level=[80,95]), cv has these columns:
          #   PatchTST          → median (50th pct)
          #   PatchTST-lo-80    → 10th pct
          #   PatchTST-hi-80    → 90th pct
          #   PatchTST-lo-95    → 2.5th pct
          #   PatchTST-hi-95    → 97.5th pct

          # Only trade when the 80% interval doesn't straddle zero (high confidence)
          confident_bull = cv_target['PatchTST-lo-80'] > 0
          confident_bear = cv_target['PatchTST-hi-80'] < 0
          has_signal = confident_bull | confident_bear

          pred_dir = np.sign(cv_target['PatchTST-median'])

          if has_signal.sum() == 0:
              fold_hit_rates.append(0.5)  # no signal = random
          else:
              hit_rate = (actual_dir[has_signal] == pred_dir[has_signal]).mean()
              fold_hit_rates.append(hit_rate)

          trial.report(np.mean(fold_hit_rates), step=fold)
          if trial.should_prune():
              raise optuna.exceptions.TrialPruned()

    return np.mean(fold_hit_rates)


study = optuna.create_study(
    direction='maximize',  # higher hit rate = better
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=1,
    )
)
study.optimize(objective, n_trials=50, timeout=3600)

print("Tickers:", tickers)
print("Period:", period_train)
print("Best params:   ", study.best_params)
print("Best hit rate: ", study.best_value)

df_trials = study.trials_dataframe()
print(df_trials.sort_values('value', ascending=False).head(10))

# Save report
with open(rdir / "optuna_summary.txt", "w") as reportfile:
    print("Tickers:",    tickers,            file=reportfile)
    print("Period:",     period_train,             file=reportfile)
    print("Best params:", study.best_params, file=reportfile)
    print("Best hit rate:", study.best_value, file=reportfile)

df_trials.sort_values("value", ascending=False).to_csv(rdir / "optuna_trials.csv", index=False)

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_steps=500` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs 

KeyError: 'PatchTST'